In [1]:
from typing import List

import torch
import torch.nn as nn
import argparse
import numpy as np
import time
import csv
from data import load_data
# from deep_knowledge_tracing_model import DeepKnowledgeTracing

import datetime
import os
import torch.utils.data as Data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from tensorboardX import SummaryWriter

from ESF_iDKT import RNN

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import metrics
from sklearn.metrics import r2_score

from tensorboardX import SummaryWriter

In [2]:
index=2
parser = argparse.ArgumentParser(description='DKT_LSTM model')
parser.add_argument('-epsilon', type=float, default=0.1, help='Epsilon value for Adam Optimizer')
parser.add_argument('-l2_lambda', type=float, default=0.3, help='Lambda for l2 loss')
parser.add_argument('-learning_rate', type=float, default=0.001, help='Learning rate')
parser.add_argument('-max_grad_norm', type=float, default=20, help='Clip gradients to this norm')
parser.add_argument('-keep_prob', type=float, default=0.6, help='Keep probability for dropout')
parser.add_argument('-hidden_layer_num', type=int, default=1, help='The number of hidden layers')
parser.add_argument('-concept_num', type=int, default=5, help='The number of concept')
parser.add_argument('-evaluation_interval', type=int, default=5, help='Evalutaion and print result every x epochs')
parser.add_argument('-batch_size', type=int, default=8, help='Batch size for training')
parser.add_argument('-epochs', type=int, default=100, help='Number of epochs to train')
parser.add_argument('-allow_soft_placement', type=bool, default=True, help='Allow device soft device placement')
parser.add_argument('-log_device_placement', type=bool, default=False, help='Log placement ofops on devices')
# parser.add_argument('-train_data_path', type=str, default='./遗忘_学生500题目60概念数5/答题序列数据_train'+str(index)+'.csv', help='Path to the training dataset')
# parser.add_argument('-test_data_path', type=str, default='./遗忘_学生500题目60概念数5/答题序列数据_test'+str(index)+'.csv',help='Path to the testing dataset')
# parser.add_argument('-qm_data_path', type=str, default='./遗忘_学生500题目60概念数5/题目知识点数据'+str(index)+'.csv',help='path to read q_matrix')
parser.add_argument('-train_data_path', type=str, default='./datasets/atc_datasets/s500_q10_kc5/s500_q10_kc5_train0.csv', help='Path to the training dataset')
parser.add_argument('-test_data_path', type=str, default='./datasets/atc_datasets/s500_q10_kc5/s500_q10_kc5_test0.csv',help='Path to the testing dataset')
# parser.add_argument('-qm_data_path', type=str, default='./qm_data/jtr/Meiosis_qm4.npy',help='path to read q_matrix')
parser.add_argument('-qm_data_path', type=str, default='./datasets/atc_datasets/s500_q10_kc5/qm_atc2.npy',help='path to read q_matrix')
parser.add_argument('-origin_DKT', type=str, default=True,help='Path to the testing dataset')
parser.add_argument('-hidden_size', type=int, default=10, help='The number of hidden nodes')

args = parser.parse_known_args()[0]

In [3]:
def run_epoch(auc_file, m, optimizer, students, batch_size, num_steps, num_skills, q_matrix, state_writer, training=True, epoch=1):
    """Runs the model on the given data."""
    #读取参数
    lr = args.learning_rate # learning rate
    train_path = args.train_data_path
    concept_num = args.concept_num
    hidden_size = args.hidden_size
    #用于统计标签计算AUC和loss
    actual_labels = []
    pred_labels = []
    total_loss = 0
    total_auc = 0
    #分批次处理数据
    index = 0
    batch_num = 0
    while(index+batch_size < len(students)):
        batch_num+=1
        #x为输入的答题数据，target_id为要预测的答题题目编号数据，不足的补齐-1
        x = np.zeros((batch_size,num_steps))
        p = np.zeros((batch_size, num_steps))
        target_id = np.ones((batch_size, num_steps))
        target_id = -target_id
        #result为输入的答题结果数据，target_correctness为预测的答题结果
        result = np.zeros((batch_size,num_steps))
        target_correctness = []
        
        answer_print=[]
        
        for i in range(batch_size):
            #student的格式：(['4'], ['77', '77', '32', '32'], ['1', '0', '0', '0'])
            student = students[index+i]
            #problem_ids:该学生答题序列的题目编号列表
            problem_ids = student[1]
            #correctness:该学生答题序列的结果列表
            correctness = student[2]
            tmp_print = []
            for l in range(len(problem_ids)):
                tmp_print.append('('+str(problem_ids[l])+','+str(correctness[l])+')')
            answer_print.append(tmp_print)
            #答题序列的最后一个作为预测，前面的作为输入
            for j in range(len(problem_ids)-1):
                #生成输入数据
                x[i,j] = int(problem_ids[j])
                result[i,j] = int(correctness[j])
                p[i,j] = int(problem_ids[j+1])
                target_id[i,j] = int(problem_ids[j+1])
                target_correctness.append(int(correctness[j+1]))
                actual_labels.append(int(correctness[j+1]))
        index += batch_size
        
        #将得到的输入训练答题数据转化为tensor，并展开得到onehot格式
        x = torch.tensor(x, dtype = torch.int64)
        x = torch.unsqueeze(x, 2)
        q_hot = torch.FloatTensor(batch_size, num_steps, num_skills)
        q_hot.zero_()
        q_hot.scatter_(2,x,1)
        #将得到的输入预测题号转化为tensor，并展开得到onehot格式
        target_id = torch.tensor(target_id, dtype = torch.int64)
        target_id = torch.unsqueeze(target_id, 2)
        p = torch.tensor(p, dtype = torch.int64)
        p = torch.unsqueeze(p, 2)
        next_q_hot = torch.FloatTensor(batch_size, num_steps, num_skills)
        next_q_hot.zero_()
        next_q_hot.scatter_(2,p,1)
        #将训练的答题结果数据转化为tensor
        result = torch.tensor(result, dtype = torch.int64)
        target_correctness = torch.tensor(target_correctness, dtype=torch.float)

        #生成mask矩阵，用于将区分有效数据和无效数据，计算loss时只考虑有效数据
        mask=[]
        label = target_id
        label = label.view(-1)
        for k in range(label.shape[0]):
            if label[k] == -1:
                mask.append(0)
            else:
                mask.append(1)
        mask = torch.tensor(mask).cuda()
        mask = mask.byte()
#         print("q_hot.shape:",q_hot.shape)
#         print("result.shape:",result.shape)
#         print("next_q_hot.shape:",next_q_hot.shape)
        #开始训练
        q_hot.cuda()
        result.cuda()
        next_q_hot.cuda()
        target_correctness.cuda()
        if training:
            optimizer.zero_grad()
            out = m(q_hot, result, next_q_hot, concept_num)
            #out的格式：(output_pred，h_t)
            output_pred = out[0]
            #利用mask矩阵从预测结果中取出有效数据
            output_pred = output_pred.view(-1)
            output_pred = torch.masked_select(output_pred, mask)
            
            #根据预测结果计算loss 
            logits = output_pred.cpu()
            preds = torch.sigmoid(logits)
            criterion = nn.BCEWithLogitsLoss()
            loss = criterion(logits, target_correctness)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            for p in preds:
                pred_labels.append(p.item())
            total_loss += loss.item()
        else:
            with torch.no_grad():
                out = m(q_hot, result, next_q_hot, concept_num)
                output_pred = out[0]
                #利用mask矩阵从预测结果中取出有效数据
                output_pred = output_pred.view(-1)
                output_pred = torch.masked_select(output_pred, mask)
                logits = output_pred.cpu()
                preds = torch.sigmoid(logits)
                criterion = nn.BCEWithLogitsLoss()
                loss = criterion(logits, target_correctness)
                for p in preds:
                    pred_labels.append(p.item())
                total_loss += loss.item()
#                 刻画学生能力变化曲线
                stu_state = out[1]
#                 for n in range(0,3):
# #                     print(len(batch_meta))
# #                     print(len(batch_meta[n]))
#                     state_writer.add_embedding(
#                         mat = stu_state[n],
#                         metadata = batch_meta[n],
#                         global_step = batch_num*batch_size+n,
#                         tag = str(epoch)
#                     )
                
                for s in range(stu_state.shape[0]):
#                     print(str(batch_num*batch_size+s))
                    ans_len = len(answer_print[s])
#                     print(answer_print[s])
                    state_writer.add_embedding(
                    mat=torch.sigmoid(stu_state[s][:ans_len,:]),#学生的能力状态向量（学生数量*hidden_size）每行表示一个数据
#                     metadata=time_list,#时间编号，对应时间步num_steps
#                     metadata=input_data[i].numpy().tolist(),
                    metadata = answer_print[s],
                    global_step=(batch_num-1)*batch_size+s,#第几个学生
                    tag=str(epoch) #第几个轮次
                )
        fpr, tpr, thresholds = metrics.roc_curve(actual_labels, pred_labels, pos_label=1)
        auc = metrics.auc(fpr, tpr)
        total_auc += auc
        r2 = r2_score(actual_labels, pred_labels)
    average_auc = total_auc/batch_num
    average_loss = total_loss/batch_num
    print("Epoch: {},   average_AUC: {}, average_loss:{}".format(epoch,  average_auc,average_loss))
    with open(auc_file, 'a') as file:
        string_auc = "Epoch:"+str(epoch)+"  average_AUC:"+str(average_auc)+"  average_loss:"+str(average_loss)+"\n"
        file.write(string_auc)
    return average_auc,average_loss,actual_labels,pred_labels          

In [4]:
def main():
    train_data_path = args.train_data_path
    test_data_path  = args.test_data_path
    batch_size = args.batch_size
    concept_num = args.concept_num
    qm_data_path = args.qm_data_path
    hidden_size = args.hidden_size
    
    
    torch.cuda.set_device(1)
    #读取数据
    train_students, train_max_num_problems, train_max_skill_num = load_data(train_data_path)
#     num_steps = train_max_num_problems
#     num_skills = train_max_skill_num
    num_layers = 3
    test_students, test_max_num_problems, test_max_skill_num = load_data(test_data_path)
    num_steps = max(train_max_num_problems, test_max_num_problems)
    num_skills = max(train_max_skill_num, test_max_skill_num)
    #读取q-matrix
#     q_matrix = np.loadtxt(qm_data_path,delimiter=',')
    q_matrix = np.load(qm_data_path)
    q_matrix = torch.from_numpy(q_matrix)
    q_matrix = torch.tensor(q_matrix, dtype = torch.float)
    q_matrix.cuda()
    print("q_matrix.shape:",q_matrix.shape)
#     print(q_matrix)
    
    print("hhm print---最大时间步长num_steps=",num_steps)
    print("hhm print---题目数量num_skills=",num_skills)
    args.origin_DKT = False
    model = RNN(num_skills,concept_num,num_skills,hidden_size,num_layers)

    #初始化kc层权重为q-matrix
    model.kc.weight = torch.nn.Parameter(q_matrix.t().cuda())
    model = torch.nn.DataParallel(model,device_ids=[1])
    model.cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, eps=args.epsilon)
    optimizer=torch.optim.Adam(model.parameters(),lr=args.learning_rate)
    now=datetime.datetime.now().replace(microsecond=0) 
    time_now = (now + datetime.timedelta(hours=8))
#     state_writer = SummaryWriter('stu_state/'+'DKT_a_kc30_'+str(num_layers)+'层')
    state_writer = SummaryWriter('stu_state/'+'m7_e5_s500_q10_kc'+str(concept_num)+'_h'+str(num_layers)+'层')
    auc_file = "result/m7_e5_s500_q10_kc-"+str(concept_num)+"_h-"+str(hidden_size)+".txt"
    test_epoch=0
    for i in range(args.epochs):
        auc,loss,actual_labels,pred_labels = run_epoch(auc_file,model, optimizer,  train_students, batch_size, num_steps, num_skills, q_matrix, state_writer, epoch=i)    

#         kc_weight = model.module.kc.weight.cpu().detach().numpy()
#         print(len(kc_weight))
        # Testing
        if ((i + 1) % args.evaluation_interval == 0):
            auc,loss,actual_labels,pred_labels = run_epoch(auc_file,model, optimizer, test_students, batch_size, num_steps, num_skills, q_matrix, state_writer, training=False, epoch=test_epoch)
            print('Testing')
            kc_weight = model.module.kc.weight.cpu().detach().numpy()
#             np.save('./atc1_kc_weight_h10/atc1_kcweight_'+str(test_epoch)+'.npy', kc_weight)
            test_epoch+=1
            # print(rmse, auc, r2)

In [5]:
import warnings

warnings.filterwarnings('ignore')
main()

the number of rows is 1200
The number of students is  400
Finish reading data
the number of rows is 300
The number of students is  100
Finish reading data
q_matrix.shape: torch.Size([10, 5])
hhm print---最大时间步长num_steps= 27
hhm print---题目数量num_skills= 10
Epoch: 0,   average_AUC: 0.6147361335208912, average_loss:0.6404476773982145
Epoch: 1,   average_AUC: 0.618649653271812, average_loss:0.6170697394682436
Epoch: 2,   average_AUC: 0.6528048780010961, average_loss:0.6005861892992136
Epoch: 3,   average_AUC: 0.7088022154111351, average_loss:0.5786026539851208
Epoch: 4,   average_AUC: 0.7297602576684428, average_loss:0.5588855220347034
Epoch: 0,   average_AUC: 0.7052532616370253, average_loss:0.5710770313938459
Testing
Epoch: 5,   average_AUC: 0.7397163085690943, average_loss:0.5478881190017778
Epoch: 6,   average_AUC: 0.7439751857742655, average_loss:0.5430096357452626
Epoch: 7,   average_AUC: 0.7460951936314087, average_loss:0.5407640027756594
Epoch: 8,   average_AUC: 0.7470873668924038, a